## Map/Reduce Semantics and Systems

### Types and transformations
* Map is a transformation
  * Input domain to output domain
* Reduce is a collection
  * No domain change
  
$$
map (k1, v1) \rightarrow list(k2,v2) \\
reduce (k2, list(v2)) \rightarrow list(v2)
$$

* Google C++ implementation is based all on strings
  * User code must convert to structured types
* Hadoop! has type wrappers

### Parallelism in Map/Reduce

* How much potential paralleism in mappers? in reducers?


.

.

......spoiler alert.......
  
.  

.
* Mappers: up to a parallel process for each input (typically a file)
* Reducers: up to a parallel process for each key
* So for the WordCount example
  * two files = two mappers
  * 5 different words = five reducers
  * but this is scalable with input


Differentiating betweens mapper/reducer and map/reduce processes
* A cluster will typically configure number of available phyical processes
  * this number is typically much smaller than potential parallelism
  * we refer to `mappers` as potential parallelism and `map processes` as the number of phyical processes running map functions.

## MR Semantics

Let's start with some definitions:

#### Shuffle

This is the routing process of mapper outputs to reducer inputs.

<img src="./images/hadoopsem.png" width=512 />

#### Partition

* Partition is the output file of a reducer _process_ (not a single reducer).
  * Contains many reducer keys
 
#### Combiner

This image is linked from https://data-flair.training/blogs/hadoop-combiner-tutorial/.  Please refer to their page.

<img src="images/combiner.jpeg" width=512 />

Combiner is a function that runs on the outputs of the mapper before the shuffle.

* Combiner executes on the mappers $ \langle key,value \rangle$ output while in memory at the mapper
  * It is possible to write unique combiner and reduce classes
  * It is common to use the reducer as a combiner
  * Combiner must have algebraic propertics, i.e. `reduce(combine(A),combine(B)) == reduce(A,B)` 
  * Traditional reduce operators (aggregates and extrema) work in combiners
* Combiner in WordCount:
  * compute sum output by mapper for each key and send a single aggregated value to reducers
* Combiner for Maximum: 
  * compute maximum value for each key.  
  * Reducer computes a maximum of maxima.

### Map/Reduce Runtimes

From the Google paper https://www.usenix.org/legacy/events/osdi04/tech/dean.html

<img src="./images/mr.png" width=512 />

* Automatically partition input data
  * 16-64 MB chunks for Google
* Create M map tasks: one for each chunk
  * Assign available workers (up to M) to tasks
* Write intermediate pairs to local (to worker) disk
* R reduce tasks (defined by user) read and process intermediate results
* Output is up to R files available on shared file system
* Master tracks state
  * Asssignment of M map tasks and R reduce tasks to workers
  * State and liveliness of the above


### Systems Issues

The map/reduce runtime must deal with:
* Master failure
  * Checkpoint/restart, classic distributed systems/replication problem
* Failed worker
  * Heartbeat liveness detection, restart
* Slow worker
  * Backup tasks
* Locality of processing to data
  * Big deal, they don’t really solve
  * But, much subsequent research does
* Task granularity
  * Metadata size and protocol scaling (not inherent parallelism) limit the size of M and R